In [1]:
from dumpr.common import BatchReader

import attr
import elasticsearch as es
from tqdm import tqdm

In [2]:
XML_SPIEGEL = '/mnt/maehre/dump/text/de/spiegel-9495/spiegel-9495.full.xml'
XML_FRA = '/mnt/maehre/dump/text/de/frankfurter-rundschau-9495/frankfurter-rundschau-9495.full.xml'

CTX = {'index': 'articles', 'doc_type': 'doc'}

In [9]:
e = es.Elasticsearch([{ 'host': 'localhost', 'port': 9200 }])
print('ping', e.ping())
    
if e.indices.exists(index=CTX['index']):
    res = e.indices.delete(index=CTX['index'])
    print('delete index:', res)

article_mapping = {
    'mappings': {
        'doc': {
            'properties': {
                'content': {'type': 'text', 'analyzer': 'german'},
                'title': {'type': 'text'},
            }
        }
    }
}

res = e.indices.create(index=CTX['index'], body=article_mapping)
print('re-create index:', res)

ping True
delete index: {'acknowledged': True}
re-create index: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'articles'}


In [10]:
def index(xml: str, total: int):
    with BatchReader(xml) as reader:
        for count, doc in tqdm(enumerate(reader), total=total):
            doc_id = doc.meta['id']
            rep = {'content': doc.content}
            
            if 'title-0' in doc.meta:
                rep['title'] = doc.meta['title-0']
            
            e.index(id=doc_id, body=rep, **CTX)
        
        print('indexed', count + 1, 'documents')

print('indexing spiegel-9495')
index(XML_SPIEGEL, 13781)

print('indexing frankfurter-rundschau-9495')
index(XML_FRA, 139715)

  0%|          | 1/13781 [00:00<24:08,  9.51it/s]

indexing spiegel-9495


  0%|          | 12/139715 [00:00<20:16, 114.85it/s]

indexed 13781 documents
indexing frankfurter-rundschau-9495


100%|██████████| 139715/139715 [14:14<00:00, 163.55it/s]

indexed 139715 documents


In [7]:
e.get(id='SPIEGEL9495-007071', **CTX)

{'_index': 'articles',
 '_type': 'doc',
 '_id': 'SPIEGEL9495-007071',
 '_version': 1,
 'found': True,
 '_source': {'content': '28, amerikanische Bürgerin, die gegen US-Präsident Bill Clinton wegen sexueller Belästigung klagt, mußte jetzt zwei juristische Niederlagen hinnehmen. Ein New Yorker Richter gestattete einem Herrenmagazin, ein Foto der entblößten Paula Jones, das sie in Umarmung mit einem Nackten zeigt, auf dem Titelblatt abzubilden. Eine Bundesrichterin in Clintons Heimatstaat Arkansas entschied, ein Prozeß gegen den Regierungschef käme frühestens nach dem Ende seiner Amtszeit in Frage. Die Klage sei offenbar nicht so dringlich: Paula Jones habe sie erst zwei Tage vor der Verjährung eingereicht.',
  'title': 'Paula Jones,'}}

In [8]:
e.get(id='FR941030-003053', **CTX)

{'_index': 'articles',
 '_type': 'doc',
 '_id': 'FR941030-003053',
 '_version': 1,
 'found': True,
 '_source': {'content': 'SSV ratiopharm Ulm SV Tally Oberelchingen 103:57 (53:25). - Beste Werfer: Knoerr Krefelder EV Düsseldorfer EG 2:4 (0:1, 1:1, 1:2) Tore: 0:1 Kummer (16:19), 0:2 Hiemer'}}